In [200]:
import pymongo
from pymongo import MongoClient
from bson import json_util
import json
import pprint
import datetime
from pytz import timezone
import pytz
import bson
from bson.objectid import ObjectId
import time
from textblob import TextBlob
import twitter_credentials
import oauth2 as oauth


In [ ]:
consumer=oauth.Consumer(key=twitter_credentials.CONSUMER_KEY,secret=twitter_credentials.CONSUMER_SECRET)
access_token=oauth.Token(key=twitter_credentials.ACCESS_TOKEN,secret=twitter_credentials.ACCESS_TOKEN_SECRET)
client=oauth.Client(consumer,access_token)

In [163]:
client=MongoClient()
db=client['tweetsdb']
tweets=db['tweets']
# tweets=db['tweets17']

In [231]:
with open('data/realdonaldtrump_2019.json') as f:
    file_data = json.load(f)

In [262]:

id=file_data[0]["id_str"]
id

'1080457560291987457'

In [164]:
#thess function updates(adds to the collection) if necessary
def insertupdate(id_t,tweet_date):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts":tweet_date}})
    except Exception as e:
        print(e)
    else:
        print("no errors")
#     tweets.insert_one({})



# def convertoint(id_t,tweet_date):
#     try:
#         tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.Day":tweet_date}})
#     except Exception as e:
#         print(e)

def insertime(id_t,tweet_time):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.time":tweet_time}})
    except Exception as e:
        print(e)
        
        
#update (2/13/2019) hours mins sec not necessary
# def insert_time_parts(id_t,hour_time):
#     try:
#         tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.hour":int(hour_time)}})
#     except Exception as e:
#         print(e)
        
        
#adds minutes as a seprate field 
#update (2/13/2019) mins sec not necessary
# def insert_time_min(id_t,min_time):
#     try:
#         tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.mins":int(min_time)}})
#     except Exception as e:
#         print(e)
        
# def insert_time_sec(id_t,sec_time):
#     try:
#         tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.sec":int(sec_time)}})
#     except Exception as e:
#         print(e)
        
        
def insert_time_stamp(id_t,time_stamp):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.timestamp":int(time_stamp)}})
    except Exception as e:
        print(e)
        
# insert a calendar format date into documents
def insert_calendar_format(id_t,calendar):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"created_at_parts.date":calendar}})
    except Exception as e:
        print(e)
        
#insert polarity value into documents
def insert_polarity(id_t,val_pol):    
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"polarity":round(val_pol,4)}})
    except Exception as e:
        print(e)
        
        
#insert subjective value into documents
def insert_subjectivity(id_t,val_subj):
    try:
        tweets.update_one({"_id":ObjectId(f"{id_t}")},{"$set":{"subjectivity":round(val_subj,4)}})
    except Exception as e:
        print(e)        
        
    
    

In [278]:
# created_at=tweets.find({},{"created_at":1,"_id":1})
created_at=tweets.find({"created_at_parts.Year":2019,"created_at_parts.Month":"Feb"})

for twts in created_at:
    date=datetime.datetime.strptime(twts["created_at"],'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC)
    ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(twts['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
    dict_one={"Month":date.strftime("%b"),"Weekday":date.strftime("%a"),"Day":int(date.strftime("%d")),"Year":date.year}
    tweet_id=twts["_id"]
    tstamp=datetime.datetime.strptime(ts, "%Y-%m-%d %H:%M:%S").timestamp()
    insert_calendar_format(tweet_id,ts[0:10])
    insert_time_stamp(tweet_id,tstamp)
#   insert_time_hour(tweet_id,hour)
#     insert_time_min(tweet_id,minutes)
#     insert_time_sec(tweet_id,sec)    
#     insertime(tweet_id,time)
#     insertupdate(tweet_id,dict_one)


In [65]:
#testing to search for a word and how many times the word is mentioned

#search for a specific word
text_search=tweets.find({"$text": {"$search": "fake news"}})

#word count
word_count=tweets.count_documents({"$text": {"$search": "hilary clinton"}})

#counts tweets in a month
monthly_tweets=tweets.count({"created_at_parts.Day":"Sep"})

print(monthly_tweets)
# for search in text_search:
#     pprint.pprint(search["full_text"])
    

281


C:\Users\alnit\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  # Remove the CWD from sys.path while we load stuff.


In [164]:
data()

In [111]:
result="2018-12-1"

date_search=tweets.find({"created_at_parts.date":{"$gt":result}})
pprint.pprint(date_search)

In [276]:
#Setting TextBlob to Each tweet
# results=tweets.find({},{"full_text":1,"_id":1})
results=tweets.find({"created_at_parts.Year":2019,"created_at_parts.Month":"Feb"})
for d in results:
    analysis=(TextBlob(d["full_text"]))
    tweet_id=d["_id"]
    polarity=analysis.sentiment[0]
    subjectivity=analysis.sentiment[1]
    insert_subjectivity(tweet_id,subjectivity)
    insert_polarity(tweet_id,polarity)

In [154]:


# results=db.tweets.aggregate([
#     {"$match":{"$text":{"$search":"hillary"}}}, 
 
#     {"$group":{
#           "_id":{"year":"$created_at_parts.Year","month":"$created_at_parts.Month"}, 
#           "count":{"$sum":1}
#     }}
# ])


In [235]:
results=tweets.find({"id_str":id})

In [242]:
# for d in results:
#     pprint.pprint(d)

In [239]:
mn=0.2
mx=0.3
results=tweets.aggregate([{"$match":{"polarity":{"$gte":mn,"$lte":mx}}}])

In [243]:
# for d in results:
#     pprint.pprint(d)

In [246]:
pol=tweets.aggregate([{"$match":{"created_at_parts.Year":2019}}])

In [248]:
# for d in pol:
#     pprint.pprint(d)